In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from sklearn import preprocessing
from itertools import combinations
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data/assignment_data.csv', sep=';')

In [3]:
data.shape

(2572, 55)

In [4]:
ids = ['country_id','application_id','product_id','customer_id']
dates = [
    'due_date',
    'first_status_day_date',
    'first_status_time_of_day',
    'paid_date',
    'arrived_date',
    'Variable_42',
    'Variable_43',
    'Variable_44']

categoricals = ['Variable_5','Variable_6','Variable_12','Variable_45']

ordinals = ['Variable_13','Variable_14']
numericals = [
    x for x in data.columns if 
    (x not in ids) and 
    (x not in dates) and 
    (x not in categoricals) and 
    (x not in ordinals) and 
    x != 'Target']

# Categorical and Ordinals

In [5]:
def transform_categorical(dataset, columns_to_transform):
    """Translates categorical columns into integers."""

    categorical_dictionary = {}
    for column in columns_to_transform:
        categories = pd.Categorical(dataset[column])
        new_column = '{prefix}{suffix}'.format(prefix=column, suffix='_INT')
        dataset.loc[:, new_column] = categories.codes
        categorical_dictionary[column] = dict([(k, v) for v, k in enumerate(categories.categories)])
        
    return dataset, categorical_dictionary

def transform_ordinals(dataset, columns_to_transform, mapper):
    """Translates categorical columns into integers."""

    categorical_dictionary = {}
    for column in columns_to_transform:
        new_column = '{prefix}{suffix}'.format(prefix=column, suffix='_INT')
        dataset.loc[:, new_column] = dataset[column].map(mapper[column])
    return dataset

ordinal_dictionary ={
    'Variable_13': {
        'A': 0,
        'B': 1,
        'C': 2,
        'D': 3,
        'E': 4,
        'F': 5,
        'G': 6,
        'H': 7,
        'I': 8,
        'J': 9,
        'K': 10,
        'L': 11,
        'M': 12},
        
    'Variable_14': {
        'RATINGSTUFE A': 0,
        'RATINGSTUFE B': 1,
        'RATINGSTUFE C': 2,
        'RATINGSTUFE D': 3,
        'RATINGSTUFE E': 4,
        'RATINGSTUFE F': 5,
        'RATINGSTUFE G': 6,
        'RATINGSTUFE H': 7,
        'RATINGSTUFE I': 8,
        'RATINGSTUFE J': 8,
        'RATINGSTUFE K': 9,
        'RATINGSTUFE L': 10,
        'RATINGSTUFE M': 11}
    }

In [6]:
data, categorical_dictionary = transform_categorical(data, categoricals)
data =  transform_ordinals(data, ordinals, ordinal_dictionary)

In [7]:
data.shape

(2572, 61)

# Dates

In [8]:
for date in dates:
    data[date] = pd.to_datetime(data[date])

In [9]:
def transform_dates_day(dataset, columns_to_transform):
    """Translates dates columns into integers. Granularity: Year, Month, Day"""
    
    for column in columns_to_transform:
        new_column_year = '{prefix}{suffix}'.format(prefix=column, suffix='_YEAR')
        new_column_month = '{prefix}{suffix}'.format(prefix=column, suffix='_MONTH')
        new_column_day = '{prefix}{suffix}'.format(prefix=column, suffix='_DAY')
        
        dataset.loc[:, new_column_year] = dataset.loc[:, column].dt.year
        dataset.loc[:, new_column_month] = dataset.loc[:, column].dt.month
        dataset.loc[:, new_column_day] = dataset.loc[:, column].dt.day
        
    return dataset

In [10]:
def transform_dates_second(dataset, columns_to_transform):
    """Translates dates columns into integers. Granularity: Hour, Minute, Second"""
    
    for column in columns_to_transform:
        new_column_hour = '{prefix}{suffix}'.format(prefix=column, suffix='_HOUR')
        new_column_minute = '{prefix}{suffix}'.format(prefix=column, suffix='_MINUTE')
        new_column_second = '{prefix}{suffix}'.format(prefix=column, suffix='_SECOND')
        
        dataset.loc[:, new_column_hour] = dataset.loc[:, column].dt.hour
        dataset.loc[:, new_column_minute] = dataset.loc[:, column].dt.minute
        dataset.loc[:, new_column_second] = dataset.loc[:, column].dt.second
        
    return dataset

In [11]:
data = transform_dates_day(data, ['due_date','first_status_day_date','paid_date','arrived_date','Variable_42','Variable_43','Variable_44'])
data = transform_dates_second(data, ['first_status_time_of_day','arrived_date'])

In [12]:
data.shape

(2572, 88)

### Feature Engineering: Adding distance between dates 

In [13]:
for couple in combinations(['due_date','first_status_day_date','paid_date','arrived_date','Variable_42','Variable_43','Variable_44'],2):
    columnn_name = "DAYS_{}-{}".format(couple[0],couple[1])
    days = (data[couple[0]] - data[couple[1]]).dt.days
    data[columnn_name] = days

In [14]:
data.shape

(2572, 109)

# Drop Transformed Variables

In [15]:
data = data.drop(categoricals, axis=1)
data = data.drop(ordinals, axis=1)
data = data.drop(dates, axis=1)

# Drop Constant Columns

In [16]:
def drop_constant_column(dataframe):
    """Drops constant value columns of pandas dataframe."""
    
    return dataframe.loc[:, (dataframe != dataframe.iloc[0]).any()]

In [17]:
data = drop_constant_column(data)

In [18]:
data.shape

(2572, 90)

In [19]:
for c in data.columns:
    if c in ids:
        print(c)

application_id
customer_id


# Drop Empty Columns  

In [20]:
empty_columns_to_drop = []
empty_threshold_to_be_dropped = 90
for column in [x for x in data.columns if x != 'Target']:
    emptyness = float(data[column].isnull().sum() * 100) /data.shape[0]
    if emptyness >= empty_threshold_to_be_dropped:
        empty_columns_to_drop.append(column)

In [21]:
data = data.drop(empty_columns_to_drop, axis=1)

In [22]:
print('Dropping {} columns'.format(len(empty_columns_to_drop)))

Dropping 24 columns


In [23]:
data.shape

(2572, 66)

# Save datasets before inputing NaN for XGBoost

In [24]:
temp_data = data[data.Target.notnull()]
target = temp_data['Target']

X_train, X_test, y_train, y_test = train_test_split(
    temp_data.drop(['Target'], 1),
    target, 
    test_size=0.25, 
    random_state=2019,
    stratify = target)

In [25]:
#data.to_csv('data/preprocessed_entire_set.csv', sep=';', index=False)
#target.to_csv('data/entire_target.csv', sep=';', index=False, header='Target')

X_train.to_csv('data/preprocessed_train_set_NAN.csv', sep=';', index=False)
y_train.to_csv('data/train_target_NAN.csv', sep=';', index=False, header='Target')

X_test.to_csv('data/preprocessed_test_set_NAN.csv', sep=';', index=False)
y_test.to_csv('data/test_target_NAN.csv', sep=';', index=False,  header='Target')

# Inputing NaN

In [26]:
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Emptyness Percentage'])
missing_data.head(10)

,Total,Emptyness Percentage
Variable_21,1990,0.773717
Variable_38,1990,0.773717
Variable_22,1757,0.683126
Variable_39,1757,0.683126
Variable_37,1100,0.427683
Variable_20,1100,0.427683
Target,515,0.200233
Variable_26,184,0.071540
Variable_27,184,0.071540
Variable_28,184,0.071540


In [27]:
for column in [x for x in data.columns if x != 'Target']:
    #data[column] = data[column].fillna((data[column].mean()))
    data[column] = data[column].fillna(0)

# MinMaxScale

In [28]:
data.shape

(2572, 66)

In [29]:
data.shape

(2572, 66)

# Grab Ferratum Test set

In [30]:
ferratum_preprocessed_test_set = data[data.Target.isnull()]

In [31]:
ferratum_preprocessed_test_set.shape

(515, 66)

In [32]:
ferratum_preprocessed_test_set.to_csv('data/ferratum_preprocessed_test_set.csv', sep=';', index=False)

# Split in Train Test

In [33]:
data = data[data.Target.notnull()]

In [34]:
target = data['Target']

In [35]:
data.drop(['Target'], 1 ,inplace=True)

In [36]:
data.head()

,application_id,Variable_1,Variable_2,Variable_4,Variable_7,Variable_8,customer_id,Variable_9,Variable_10,Variable_11,...,first_status_time_of_day_MINUTE,first_status_time_of_day_SECOND,arrived_date_HOUR,arrived_date_MINUTE,DAYS_due_date-first_status_day_date,DAYS_due_date-paid_date,DAYS_due_date-arrived_date,DAYS_first_status_day_date-paid_date,DAYS_first_status_day_date-arrived_date,DAYS_paid_date-arrived_date
0,24176,35,1,1,27.0,100,2970192,190,100,100,...,43,22,0,43,182,151,181,-31,-1,30
1,24185,30,1,1,26.0,199,2970195,199,199,199,...,56,36,7,56,32,1,31,-31,-1,30
2,24215,30,1,1,21.0,100,2970204,100,100,100,...,2,4,11,2,32,1,31,-31,-1,30
3,24220,30,1,1,48.0,199,2970206,199,199,199,...,22,37,11,22,192,30,191,-162,-1,161
4,24255,30,2,1,28.0,199,2430110,600,199,199,...,27,18,15,27,32,1,31,-31,-1,30


In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    data, 
    target, 
    test_size=0.25, 
    random_state=2019,
    stratify = target)

In [38]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1542, 65)
(1542,)
(515, 65)
(515,)


In [39]:
data.to_csv('data/preprocessed_entire_set.csv', sep=';', index=False)
target.to_csv('data/entire_target.csv', sep=';', index=False, header='Target')

X_train.to_csv('data/preprocessed_train_set.csv', sep=';', index=False)
y_train.to_csv('data/train_target.csv', sep=';', index=False, header='Target')

X_test.to_csv('data/preprocessed_test_set.csv', sep=';', index=False)
y_test.to_csv('data/test_target.csv', sep=';', index=False,  header='Target')